# Fixing data
Here is the process of data cleaning 

In [2]:
import pandas as pd
import re

In [3]:

df = pd.read_csv("eurocup_2020_results.csv")
df2 = pd.read_csv('players2021_complete.csv')


# working with df2: getting rid of all players not in the tournament

## insert all Euro2020 player names

In [1]:
countries=['Austria', 'Belgium', 'Croatia', 'Czech Republic', 'Denmark', 'England', 'Finland', 'France', 'Germany', 'Hungary', 'Italy', 'Netherlands', 'North Macedonia', 'Poland', 'Portugal', 'Russia', 'Scotland', 'Slovakia', 'Spain', 'Sweden', 'Switzerland', 'Turkey', 'Ukraine', 'Wales']


In [2]:
player_list=[['Daniel Bachmann',
  'Pavao Pervan',
  'Alexander Schlager',
  'David Alaba',
  'Aleksandar Dragović',
  'Marco Friedl',
  'Martin Hinteregger',
  'Stefan Lainer',
  'Philipp Lienhart',
  'Stefan Posch',
  'Andreas Ulmer',
  'Julian Baumgartlinger',
  'Christoph Baumgartner',
  'Florian Grillitsch',
  'Stefan Ilsanker',
  'Konrad Laimer',
  'Valentino Lazaro',
  'Karim Onisiwo',
  'Marcel Sabitzer',
  'Louis Schaub',
  'Xaver Schlager',
  'Alessandro Schöpf',
  'Christopher Trimmel',
  'Marko Arnautović',
  'Michael Gregoritsch',
  'Sasa Kalajdzic'],
 ['Thibaut Courtois',
  'Matz Sels',
  'Thomas Kaminski',
  'Toby Alderweireld',
  'Dedryck Boyata',
  'Timothy Castagne',
  'Jason Denayer',
  'Thomas Meunier',
  'Thomas Vermaelen',
  'Jan Vertonghen',
  'Nacer Chadli',
  'Yannick Carrasco',
  'Kevin De Bruyne',
  'Leander Dendoncker',
  'Eden Hazard',
  'Thorgan Hazard',
  'Dennis Praet',
  'Youri Tielemans',
  'Hans Vanaken',
  'Axel Witsel',
  'Michy Batshuayi',
  'Christian Benteke',
  'Jeremy Doku',
  'Romelu Lukaku',
  'Dries Mertens',
  'Leandro Trossard'],
 ['Dominik Livaković',
  'Lovre Kalinić',
  'Simon Sluga',
  'Borna Barišić',
  'Domagoj Bradarić',
  'Duje Ćaleta-Car',
  'Joško Gvardiol',
  'Josip Juranović',
  'Dejan Lovren',
  'Mile Škorić',
  'Domagoj Vida',
  'Šime Vrsaljko',
  'Mateo Kovačić',
  'Luka Modrić',
  'Marcelo Brozović',
  'Milan Badelj',
  'Nikola Vlašić',
  'Mario Pašalić',
  'Ivan Perišić',
  'Mislav Oršić',
  'Luka Ivanušec',
  'Josip Brekalo',
  'Ante Budimir',
  'Andrej Kramarić',
  'Bruno Petković',
  'Ante Rebić'],
 ['Tomáš Koubek',
  'Aleš Mandous',
  'Tomáš Vaclík',
  'Jan Bořil',
  'Jakub Brabec',
  'Ondřej Čelůstka',
  'Vladimír Coufal',
  'Pavel Kadeřábek',
  'Tomáš Kalas',
  'Tomáš Holeš',
  'Aleš Matějů',
  'David Zima',
  'Antonín Barák',
  'Vladimír Darida',
  'Jakub Jankto',
  'Alex Král',
  'Lukáš Masopust',
  'Jakub Pešek',
  'Michal Sadílek',
  'Petr Ševčík',
  'Tomáš Souček',
  'Adam Hložek',
  'Michael Krmenčík',
  'Tomáš Pekhart',
  'Patrik Schick',
  'Matěj Vydra'],
 ['Jonas Lössl',
  'Frederik Rønnow',
  'Kasper Schmeichel',
  'Joachim Andersen',
  'Nicolai Boilesen',
  'Andreas Christensen',
  'Mathias Jørgensen',
  'Simon Kjær',
  'Joakim Mæhle',
  'Jens Stryger Larsen',
  'Jannik Vestergaard',
  'Anders Christiansen',
  'Thomas Delaney',
  'Christian Eriksen',
  'Pierre-Emile Højbjerg',
  'Mathias Jensen',
  'Christian Nørgaard',
  'Robert Skov',
  'Daniel Wass',
  'Martin Braithwaite',
  'Andreas Cornelius',
  'Mikkel Damsgaard',
  'Kasper Dolberg',
  'Andreas Skov Olsen',
  'Yussuf Poulsen',
  'Jonas Wind'],
 ['Sam Johnstone',
  'Jordan Pickford',
  'Aaron Ramsdale',
  'Ben Chilwell',
  'Conor Coady',
  'Reece James',
  'Harry Maguire',
  'Tyrone Mings',
  'Luke Shaw',
  'John Stones',
  'Kieran Trippier',
  'Kyle Walker',
  'Ben White',
  'Jude Bellingham',
  'Phil Foden',
  'Jack Grealish',
  'Jordan Henderson',
  'Mason Mount',
  'Kalvin Phillips',
  'Declan Rice',
  'Bukayo Saka',
  'Jadon Sancho',
  'Dominic Calvert-Lewin',
  'Harry Kane',
  'Marcus Rashford',
  'Raheem Sterling'],
 ['Lukas Hradecky',
  'Anssi Jaakkola',
  'Jesse Joronen',
  'Paulus Arajuuri',
  'Robert Ivanov',
  'Thomas Lam',
  'Daniel OShaughnessy',
  'Jukka Raitala',
  'Joona Toivio',
  'Jere Uronen',
  'Nicholas Hämäläinen',
  'Sauli Väisänen',
  'Nikolai Alho',
  'Fredrik Jensen',
  'Glen Kamara',
  'Joni Kauko',
  'Robin Lod',
  'Rasmus Schüller',
  'Pyry Soiri',
  'Tim Sparv',
  'Robert Taylor',
  'Onni Valakari',
  'Marcus Forss',
  'Lassi Lappalainen',
  'Joel Pohjanpalo',
  'Teemu Pukki'],
 ['Hugo Lloris',
  'Mike Maignan',
  'Steve Mandanda',
  'Lucas Digne',
  'Léo Dubois',
  'Lucas Hernández',
  'Presnel Kimpembe',
  'Jules Koundé',
  'Clément Lenglet',
  'Benjamin Pavard',
  'Raphaël Varane',
  'Kurt Zouma',
  'Kingsley Coman',
  'NGolo Kanté',
  'Thomas Lemar',
  'Paul Pogba',
  'Adrien Rabiot',
  'Moussa Sissoko',
  'Corentin Tolisso',
  'Wissam Ben Yedder',
  'Karim Benzema',
  'Ousmane Dembélé',
  'Olivier Giroud',
  'Antoine Griezmann',
  'Kylian Mbappé',
  'Marcus Thuram'],
 ['Manuel Neuer',
  'Bernd Leno',
  'Kevin Trapp',
  'Emre Can',
  'Matthias Ginter',
  'Robin Gosens',
  'Christian Günter',
  'Marcel Halstenberg',
  'Mats Hummels',
  'Lukas Klostermann',
  'Robin Koch',
  'Antonio Rüdiger',
  'Niklas Süle',
  'Serge Gnabry',
  'Leon Goretzka',
  'İlkay Gündoğan',
  'Jonas Hofmann',
  'Joshua Kimmich',
  'Toni Kroos',
  'Jamal Musiala',
  'Florian Neuhaus',
  'Leroy Sané',
  'Kai Havertz',
  'Thomas Müller',
  'Kevin Volland',
  'Timo Werner'],
 ['Ádám Bogdán',
  'Dénes Dibusz',
  'Péter Gulácsi',
  'Bendegúz Bolla',
  'Endre Botka',
  'Attila Fiola',
  'Ákos Kecskés',
  'Ádám Lang',
  'Gergő Lovrencsics',
  'Loïc Négo',
  'Willi Orbán',
  'Attila Szalai',
  'Tamás Cseri',
  'Dániel Gazdag',
  'László Kleinheisler',
  'Ádám Nagy',
  'András Schäfer',
  'Dávid Sigér',
  'Kevin Varga',
  'Roland Varga',
  'János Hahn',
  'Filip Holender',
  'Nemanja Nikolić',
  'Roland Sallai',
  'Szabolcs Schön',
  'Ádám Szalai'],
 ['Gianluigi Donnarumma',
  'Alex Meret',
  'Salvatore Sirigu',
  'Francesco Acerbi',
  'Alessandro Bastoni',
  'Leonardo Bonucci',
  'Giorgio Chiellini',
  'Giovanni Di Lorenzo',
  'Emerson',
  'Alessandro Florenzi',
  'Leonardo Spinazzola',
  'Rafael Tolói',
  'Nicolò Barella',
  'Federico Bernardeschi',
  'Federico Chiesa',
  'Bryan Cristante',
  'Jorginho',
  'Manuel Locatelli',
  'Matteo Pessina',
  'Marco Verratti',
  'Gaetano Castrovilli',
  'Andrea Belotti',
  'Domenico Berardi',
  'Ciro Immobile',
  'Lorenzo Insigne',
  'Giacomo Raspadori'],
 ['Marco Bizot',
  'Tim Krul',
  'Maarten Stekelenburg',
  'Nathan Aké',
  'Daley Blind',
  'Matthijs de Ligt',
  'Stefan de Vrij',
  'Denzel Dumfries',
  'Patrick van Aanholt',
  'Joël Veltman',
  'Owen Wijndal',
  'Frenkie de Jong',
  'Marten de Roon',
  'Ryan Gravenberch',
  'Davy Klaassen',
  'Teun Koopmeiners',
  'Quincy Promes',
  'Jurriën Timber',
  'Georginio Wijnaldum',
  'Steven Berghuis',
  'Luuk de Jong',
  'Memphis Depay',
  'Cody Gakpo',
  'Donyell Malen',
  'Wout Weghorst'],
 ['Stole Dimitrievski',
  'Risto Jankov',
  'Damjan Siskovski',
  'Ezgjan Alioski',
  'Egzon Bejtulai',
  'Visar Musliu',
  'Kire Ristevski',
  'Stefan Ristovski',
  'Darko Velkovski',
  'Gjoko Zajkov',
  'Arijan Ademi',
  'Enis Bardi',
  'Darko Churlinov',
  'Eljif Elmas',
  'Ferhan Hasani',
  'Tihomir Kostadinov',
  'Boban Nikolov',
  'Milan Ristovski',
  'Stefan Spirovski',
  'Daniel Avramovski',
  'Goran Pandev',
  'Marjan Radeski',
  'Vlatko Stojanovski',
  'Aleksandar Trajkovski',
  'Ivan Tričkovski',
  'Krste Velkoski'],
 ['Łukasz Fabiański',
  'Łukasz Skorupski',
  'Wojciech Szczęsny',
  'Jan Bednarek',
  'Bartosz Bereszyński',
  'Kamil Glik',
  'Michał Helik',
  'Tomasz Kędziora',
  'Kamil Piątkowski',
  'Tymoteusz Puchacz',
  'Maciej Rybus',
  'Paweł Dawidowicz',
  'Przemysław Frankowski',
  'Kamil Jóźwiak',
  'Mateusz Klich',
  'Kacper Kozłowski',
  'Grzegorz Krychowiak',
  'Karol Linetty',
  'Jakub Moder',
  'Przemysław Płacheta',
  'Piotr Zieliński',
  'Dawid Kownacki',
  'Robert Lewandowski',
  'Karol Świderski',
  'Jakub Świerczok'],
 ['Anthony Lopes',
  'Rui Patrício',
  'Rui Silva',
  'Rúben Dias',
  'José Fonte',
  'Raphael Guerreiro',
  'Nuno Mendes',
  'Pepe',
  'Nélson Semedo',
  'Diogo Dalot',
  'William Carvalho',
  'Danilo',
  'Bruno Fernandes',
  'Gonçalo Guedes',
  'João Moutinho',
  'Rúben Neves',
  'Sérgio Oliveira',
  'João Palhinha',
  'Pote',
  'Renato Sanches',
  'Bernardo Silva',
  'João Félix',
  'Diogo Jota',
  'Cristiano Ronaldo',
  'André Silva',
  'Rafa Silva'],
 ['Yuri Dyupin',
  'Matvei Safonov',
  'Anton Shunin',
  'Igor Diveev',
  'Georgi Dzhikiya',
  'Mário Fernandes',
  'Vyacheslav Karavaev',
  'Fedor Kudryashov',
  'Andrei Semenov',
  'Roman Evgenyev',
  'Dmitri Barinov',
  'Denis Cheryshev',
  'Daniil Fomin',
  'Aleksandr Golovin',
  'Daler Kuzyaev',
  'Maksim Mukhin',
  'Magomed Ozdoev',
  'Rifat Zhemaletdinov',
  'Yuri Zhirkov',
  'Roman Zobnin',
  'Artem Dzyuba',
  'Aleksei Ionov',
  'Denis Makarov',
  'Aleksei Miranchuk',
  'Aleksandr Sobolev',
  'Anton Zabolotny'],
 ['Craig Gordon',
  'David Marshall',
  'Jon McLaughlin',
  'Liam Cooper',
  'Declan Gallagher',
  'Grant Hanley',
  'Jack Hendry',
  'Scott McKenna',
  'Stephen ODonnell',
  'Nathan Patterson',
  'Andy Robertson',
  'Greg Taylor',
  'Kieran Tierney',
  'Stuart Armstrong',
  'Ryan Christie',
  'John Fleck',
  'James Forrest',
  'Ryan Fraser',
  'Billy Gilmour',
  'John McGinn',
  'Callum McGregor',
  'Scott McTominay',
  'David Turnbull',
  'Ché Adams',
  'Lyndon Dykes',
  'Kevin Nisbet'],
 ['Martin Dúbravka',
  'Dušan Kuciak',
  'Marek Rodák',
  'Dávid Hancko',
  'Tomáš Hubočan',
  'Martin Koscelník',
  'Peter Pekarík',
  'Ľubomír Šatka',
  'Milan Škriniar',
  'Martin Valjent',
  'Denis Vavro',
  'László Bénes',
  'Ondrej Duda',
  'Ján Greguš',
  'Marek Hamšík',
  'Lukáš Haraslín',
  'Jakub Hromada',
  'Patrik Hrošovský',
  'Juraj Kucka',
  'Stanislav Lobotka',
  'Tomáš Suslov',
  'Vladimír Weiss',
  'Róbert Boženík',
  'Michal Ďuriš',
  'Róbert Mak',
  'Ivan Schranz'],
 ['David de Gea',
  'Unai Simón',
  'Robert Sánchez',
  'José Gayà',
  'Jordi Alba',
  'Pau Torres',
  'Aymeric Laporte',
  'Eric García',
  'Diego Llorente',
  'César Azpilicueta',
  'Thiago Alcántara',
  'Sergio Busquets',
  'Koke',
  'Marcos Llorente',
  'Dani Olmo',
  'Rodri',
  'Fabián Ruiz',
  'Pablo Sarabia',
  'Ferran Torres',
  'Adama Traoré',
  'Álvaro Morata',
  'Gerard Moreno',
  'Mikel Oyarzabal',
  'Pedri'],
 ['Karl-Johan Johnsson',
  'Kristoffer Nordfeldt',
  'Robin Olsen',
  'Ludwig Augustinsson',
  'Pierre Bengtsson',
  'Marcus Danielson',
  'Andreas Granqvist',
  'Filip Helander',
  'Pontus Jansson',
  'Emil Krafth',
  'Victor Lindelöf',
  'Mikael Lustig',
  'Jens-Lys Cajuste',
  'Viktor Claesson',
  'Albin Ekdal',
  'Emil Forsberg',
  'Dejan Kulusevski',
  'Sebastian Larsson',
  'Kristoffer Olsson',
  'Robin Quaison',
  'Ken Sema',
  'Mattias Svanberg',
  'Gustav Svensson',
  'Marcus Berg',
  'Alexander Isak',
  'Jordan Larsson'],
 ['Yvon Mvogo',
  'Yann Sommer',
  'Gregor Kobel',
  'Manuel Akanji',
  'Loris Benito',
  'Eray Cömert',
  'Nico Elvedi',
  'Jordan Lotomba',
  'Kevin Mbabu',
  'Becir Omeragic',
  'Ricardo Rodríguez',
  'Fabian Schär',
  'Silvan Widmer',
  'Christian Fassnacht',
  'Edimilson Fernandes',
  'Remo Freuler',
  'Xherdan Shaqiri',
  'Djibril Sow',
  'Ruben Vargas',
  'Granit Xhaka',
  'Denis Zakaria',
  'Steven Zuber',
  'Breel Embolo',
  'Mario Gavranović',
  'Admir Mehmedi',
  'Haris Seferović'],
 ['Altay Bayındır',
  'Uğurcan Çakır',
  'Mert Günok',
  'Kaan Ayhan',
  'Zeki Çelik',
  'Merih Demiral',
  'Ozan Kabak',
  'Umut Meraş',
  'Mert Müldür',
  'Çağlar Söyüncü',
  'Rıdvan Yılmaz',
  'Taylan Antalyalı',
  'Hakan Çalhanoğlu',
  'Halil Dervişoğlu',
  'İrfan Can Kahveci',
  'Orkun Kökçü',
  'Abdülkadir Ömür',
  'Dorukhan Toköz',
  'Ozan Tufan',
  'Okay Yokuşlu',
  'M. Kerem Aktürkoğlu',
  'Kenan Karaman',
  'Enes Ünal',
  'Cengiz Ünder',
  'Yusuf Yazıcı',
  'Burak Yılmaz'],
 ['Georgiy Bushchan',
  'Andriy Pyatov',
  'Anatolii Trubin',
  'Oleksandr Karavaev',
  'Serhiy Kryvtsov',
  'Mykola Matviyenko',
  'Vitaliy Mykolenko',
  'Denys Popov',
  'Eduard Sobol',
  'Oleksandr Tymchyk',
  'Illia Zabarnyi',
  'Oleksandr Zinchenko',
  'Roman Bezus',
  'Yevhen Makarenko',
  'Ruslan Malinovskyi',
  'Marlos',
  'Mykola Shaparenko',
  'Taras Stepanenko',
  'Heorhii Sudakov',
  'Serhiy Sydorchuk',
  'Viktor Tsygankov',
  'Artem Besedin',
  'Artem Dovbyk',
  'Roman Yaremchuk',
  'Andriy Yarmolenko',
  'Oleksandr Zubkov'],
 ['Adam Davies',
  'Wayne Hennessey',
  'Danny Ward',
  'Ethan Ampadu',
  'Ben Cabango',
  'Ben Davies',
  'Chris Gunter',
  'Tom Lockyer',
  'Chris Mepham',
  'Rhys Norrington-Davies',
  'Connor Roberts',
  'Joe Rodon',
  'Neco Williams',
  'Joe Allen',
  'David Brooks',
  'Rubin Colwill',
  'Daniel James',
  'Dylan Levitt',
  'Joe Morrell',
  'Aaron Ramsey',
  'Matthew Smith',
  'Jonny Williams',
  'Harry Wilson',
  'Gareth Bale',
  'Kieffer Moore',
  'Tyler Roberts']]

## filter data frame with just players from Euro2020

In [6]:
euro2020 = pd.DataFrame()

for teams in player_list:
    for players in teams:
        a=df2[(df2['name']== players)]
        
        euro2020=euro2020.append(a,ignore_index=True)

euro2020.drop(columns=['player_id','hits'], inplace=True)

In [7]:
euro2020.tail(n=1)


,name,nationality,position,overall,age,potential,team
472,Tyler Roberts,Wales,ST|CAM,70,21,81,Leeds United


In [8]:
unique_nationalities=df2['nationality'].unique()
unique_nationalities2=euro2020['nationality'].unique()
print (len(countries))
print (countries)
print(len(unique_nationalities2))
print(unique_nationalities2)
to_remove=['Mali','Ivory Coast','Australia','Brazil']

24
['Austria', 'Belgium', 'Croatia', 'Czech Republic', 'Denmark', 'England', 'Finland', 'France', 'Germany', 'Hungary', 'Italy', 'Netherlands', 'North Macedonia', 'Poland', 'Portugal', 'Russia', 'Scotland', 'Slovakia', 'Spain', 'Sweden', 'Switzerland', 'Turkey', 'Ukraine', 'Wales']
28
['Austria' 'Belgium' 'Croatia' 'Czech Republic' 'Denmark' 'England'
 'Finland' 'France' 'Germany' 'Hungary' 'Italy' 'Brazil' 'Netherlands'
 'North Macedonia' 'Poland' 'Portugal' 'Russia' 'Scotland' 'Australia'
 'Slovakia' 'Spain' 'Mali' 'Ivory Coast' 'Sweden' 'Switzerland' 'Turkey'
 'Ukraine' 'Wales']


In [9]:
for team in to_remove:
    a=euro2020[euro2020['nationality'] == team].index
    euro2020.drop(a, inplace=True)
    

In [10]:
euro2020
unique_nationalities=df2['nationality'].unique()
unique_nationalities2=euro2020['nationality'].unique()

print(len(unique_nationalities2))
print(unique_nationalities2)

24
['Austria' 'Belgium' 'Croatia' 'Czech Republic' 'Denmark' 'England'
 'Finland' 'France' 'Germany' 'Hungary' 'Italy' 'Netherlands'
 'North Macedonia' 'Poland' 'Portugal' 'Russia' 'Scotland' 'Slovakia'
 'Spain' 'Sweden' 'Switzerland' 'Turkey' 'Ukraine' 'Wales']


In [24]:
euro2020

,name,nationality,position,overall,age,potential,team
0,Daniel Bachmann,Austria,GK,69,26,75,Watford
1,Pavao Pervan,Austria,GK,73,32,73,VfL Wolfsburg
2,Alexander Schlager,Austria,GK,74,24,80,LASK Linz
3,David Alaba,Austria,LB|CB,83,28,84,FC Bayern München
4,Marco Friedl,Austria,LB|CB,68,22,75,SV Werder Bremen
...,...,...,...,...,...,...,...
468,Aaron Ramsey,Wales,CM|CAM,83,29,83,Juventus
469,Harry Wilson,Wales,RM|CM,77,23,84,AFC Bournemouth
470,Gareth Bale,Wales,ST|RW|LW,85,31,85,Real Madrid
471,Kieffer Moore,Wales,ST,70,27,71,Wigan Athletic


In [25]:
df2_json=euro2020.to_json(orient="records")

file= open('players_stats_2.json', 'w')
file.write(df2_json)
file.close()

In [18]:
df2_csv=euro2020.to_csv()

file2= open('player_stats_mdb.csv', 'w')
file2.write(df2_csv)
file2.close()

# working with df1: match stats

## tests
We will attempt to just keep the data about goal scoring using different methods

In [ ]:
ev_test=df['events_list']

ev_test[2]  #10

In [ ]:
a=ev_test[2]
a=a.split(sep='{')
print (type(a))

#to do: eliminate all info that isnt a goal or own goal, perhaps change info to say name of team instead of just away and home

In [ ]:
a

In [ ]:
# #working as intended
# for data in a:
#     if 'Goal' in data:
#         print (data,'\n')
#     if 'goal' in data:
#           print (data,'\n')

In [ ]:
#[i for i in my_list if '91' not in i and '18' not in i]
b=[data for data in a if 'goal' in data.lower() or 'Penalty' in data and 'PK' not in data and 'Disallowed' not in data]
c=[data for data in b if 'False' not in data]


In [ ]:
#x = txt.replace("thingtoremove", "subs")
#re.sub(r'\W+', '', your_string)
team=[]
minute=[]
player=[]
for data in b:
    team.append(re.findall(('away|home'),data))
    minute.append(re.search('\d+',data).group())
    if 'Own goal' in data:
        player.append(re.search('(?:action_player_1).*?\w+\s\w*',data).group().replace("action_player_1': ' ","own goal-"))
    else:
        player.append(re.search('(?:action_player_1).*?\w+\s\w*',data).group().replace("action_player_1': ' ",""))

team=[x for [x] in team]
print(type(team))
print(team)
print(minute)
print(player)

#re.sub(r'[^A-Za-z0-9 ]+', '', s)

In [ ]:
final=[(team[i],player[i],minute[i]) for i in range (len(player))]
final

In [ ]:
final_l=[]
for data in final:
    
    final_l.append(list(data))
    
final_l

## function defining and  applying


In [4]:
def pichichi(event):
    team=[]
    minute=[]
    player=[]
    final=[]
    try:
        event_=event.split(sep='{')
        

        event_2=[data for data in event_ if 'goal' in data.lower() or 'Penalty' in data and 'PK' not in data and 'Disallowed' not in data]
        event_2=[data for data in event_2 if 'False' not in data]
        event_2=[data for data in event_2 if 'Disallowed' not in data]
        

        for data in event_2:        
            team.append(re.findall(('away|home'),data))

            minute.append(re.search('\d+',data).group())

            if 'Own goal' in data:
                player.append(re.search('(?:action_player_1).*?\w+\s\w*',data).group().replace("action_player_1': ' ","own goal-"))
            else:
                player.append(re.search('(?:action_player_1).*?\w+\s\w*',data).group().replace("action_player_1': ' ",""))

        team=[x for [x] in team]   
        final=[({'team':team[i]},{'player':player[i]},{'minute':minute[i]}) for i in range (len(player))]
        final_l=[]
        for data in final:
            final_l.append(list(data))
        if final_l==[]:
            final_l=['no goals']
        return final_l
    except:
        return 'error'
    



In [5]:
df['events_list'].sample(n=1)

0    [{'event_team': 'away', 'event_time': " 2' ", ...
Name: events_list, dtype: object

###### apply f(x)

In [6]:
df['events_list']=df['events_list'].apply(pichichi)

###### after some battling, the function for filtering just the goals made during the match time (excluidng OT penalty shootout) is working as intended

In [7]:
df.head(n=1)

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,events_list,lineup_home,lineup_away
0,Final,11.07.2021,True,3,2,Italy,England,1,1,66%,34%,19,6,6,2,53%,47%,"[[{'team': 'away'}, {'player': 'Luke Shaw'}, {...","[{'Player_Name': 'Insigne', 'Player_Number': '...","[{'Player_Name': 'Kane', 'Player_Number': '9'}..."


In [8]:
df.drop(columns=['lineup_home','lineup_away'], inplace=True)

In [9]:
df=df[['stage','date','team_name_home','team_home_score','team_name_away','team_away_score','events_list' ,'pens','pens_home_score','pens_away_score','possession_home','possession_away', 'total_shots_home','total_shots_away','shots_on_target_home','shots_on_target_away']]

In [10]:
df=df.rename(columns={'events_list':'goals_scorers'})

In [11]:
df=df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [12]:
df.reset_index(inplace=True)

In [17]:
df=df.rename(columns={'index':'_id'})

In [18]:
df

,_id,stage,date,team_name_home,team_home_score,team_name_away,team_away_score,goals_scorers,pens,pens_home_score,pens_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away
0,0,Final,11.07.2021,Italy,1,England,1,"[[{'team': 'away'}, {'player': 'Luke Shaw'}, {...",True,3,2,66%,34%,19,6,6,2
1,1,Semi-finals,07.07.2021,England,2,Denmark,1,"[[{'team': 'away'}, {'player': 'Mikkel Damsgaa...",False,False,False,59%,41%,20,6,10,3
2,2,Semi-finals,06.07.2021,Italy,1,Spain,1,"[[{'team': 'home'}, {'player': 'Federico Chies...",True,4,2,29%,71%,7,16,4,5
3,3,Quarter-finals,03.07.2021,Ukraine,0,England,4,"[[{'team': 'away'}, {'player': 'Harry Kane'}, ...",False,False,False,48%,52%,7,10,2,6
4,4,Quarter-finals,03.07.2021,Czech Republic,1,Denmark,2,"[[{'team': 'away'}, {'player': 'Thomas Delaney...",False,False,False,56%,44%,16,11,5,7
5,5,Quarter-finals,02.07.2021,Belgium,1,Italy,2,"[[{'team': 'away'}, {'player': 'Nicolo Barella...",False,False,False,46%,54%,10,14,4,3
6,6,Quarter-finals,02.07.2021,Switzerland,1,Spain,1,"[[{'team': 'away'}, {'player': 'own goal-Denis...",True,1,3,27%,73%,8,28,2,10
7,7,Round of 16,29.06.2021,Sweden,1,Ukraine,2,"[[{'team': 'away'}, {'player': 'Olexandr Zinch...",False,False,False,45%,55%,13,15,3,4
8,8,Round of 16,29.06.2021,England,2,Germany,0,"[[{'team': 'home'}, {'player': 'Raheem Sterlin...",False,False,False,45%,55%,5,9,4,3
9,9,Round of 16,28.06.2021,France,3,Switzerland,3,"[[{'team': 'away'}, {'player': 'Haris Seferovi...",True,4,5,55%,45%,26,12,8,5


Some different json files will be created to assess which one fits the API. 
Unused options were commented out


In [21]:
df_csv=df.to_csv()

file_df= open('matches_mdb.csv', 'w')
file_df.write(df_csv)
file_df.close()

In [19]:
#df_json=df.to_json(orient='split')
# df_json2=df.to_json(orient="table")
# df_json3=df.to_json(orient="columns")
#df_json4=df.to_json(orient="index") 
df_json5=df.to_json(orient="records")
# df_test=df.to_csv

In [20]:
#f= open('matches.json', 'w')
#f.write(df_json)
#f.close()

# f2= open('matches.json', 'w')
# f2.write(df_json2)
# f2.close()

# f3= open('matches3_col.json', 'w')
# f3.write(df_json3)
# f3.close()

# f4= open('matches4_index.json', 'w')
# f4.write(df_json4)
# f4.close()

f5= open('matches5_rec.json', 'w')
f5.write(df_json5)
f5.close()